In [19]:
# pip install pickle-mixin, keras, tensorflow, nltk
# nltk.download('punkt')
# nltk.download('wordnet')
# %pip install autocorrect


In [1]:
import nltk
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() # onverting a word to its base form
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [2]:
# print(data_file)

In [3]:
'''Preprocess data'''

for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        w = nltk.word_tokenize(pattern) # tokenize each word
        words.extend(w) 
        
        documents.append((w, intent['tag'])) # add documents in the corpus
        
        if intent['tag'] not in classes:
            classes.append(intent['tag']) # add to our classes list
            
            
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

39 documents
8 classes ['books', 'goodbye', 'greeting', 'learn', 'music', 'options', 'shows', 'thanks']
70 unique lemmatized words ["'s", ',', 'a', 'animation', 'any', 'anyone', 'are', 'awesome', 'be', 'book', 'bye', 'can', 'chatting', 'chinese', 'could', 'day', 'do', 'draw', 'english', 'for', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'here', 'hey', 'hi', 'hola', 'how', 'i', 'is', 'later', 'learn', 'listen', 'math', 'me', 'music', 'next', 'nice', 'offered', 'patrol', 'paw', 'peppa', 'pig', 'provide', 'read', 'see', 'show', 'sing', 'some', 'someone', 'song', 'sound', 'support', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'track', 'tv', 'want', 'watch', 'what', 'you']


In [4]:
'''Creating datasets'''

# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        # output is a '0' for each tag and '1' for current tag (for each pattern)
        output_row = list(output_empty)
        output_row[classes.index(doc[1])] = 1
        training.append([bag, output_row])
    
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [5]:
'''Build NN model'''

# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/100
546/546 [==============================] - 0s 610us/step - loss: 0.6581 - accuracy: 0.7897
Epoch 2/100
546/546 [==============================] - 0s 687us/step - loss: 0.0631 - accuracy: 0.98790s - loss: 0.0644 - accuracy: 0.98
Epoch 3/100
546/546 [==============================] - 0s 722us/step - loss: 0.0265 - accuracy: 0.9956
Epoch 4/100
546/546 [==============================] - 0s 604us/step - loss: 0.0244 - accuracy: 0.9945
Epoch 5/100
546/546 [==============================] - 0s 609us/step - loss: 0.0140 - accuracy: 0.9974
Epoch 6/100
546/546 [==============================] - 0s 594us/step - loss: 0.0113 - accuracy: 0.9985
Epoch 7/100
546/546 [==============================] - 0s 580us/step - loss: 0.0095 - accuracy: 0.9982
Epoch 8/100
546/546 [==============================] - 0s 604us/step - loss: 0.0092 - accuracy: 0.9985
Epoch 9/100
546/546 [==============================] - 0s 666us/step - loss: 0.0105 - accuracy: 0.9971
Epoch 10/100
546/546 [=================

546/546 [==============================] - 0s 614us/step - loss: 0.0019 - accuracy: 0.9989
Epoch 80/100
546/546 [==============================] - 0s 727us/step - loss: 5.2131e-04 - accuracy: 1.0000
Epoch 81/100
546/546 [==============================] - 0s 809us/step - loss: 8.0355e-04 - accuracy: 0.9996
Epoch 82/100
546/546 [==============================] - 0s 583us/step - loss: 0.0015 - accuracy: 0.9993
Epoch 83/100
546/546 [==============================] - 0s 612us/step - loss: 7.0589e-04 - accuracy: 1.0000
Epoch 84/100
546/546 [==============================] - 0s 654us/step - loss: 7.9408e-04 - accuracy: 1.0000
Epoch 85/100
546/546 [==============================] - 0s 606us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 86/100
546/546 [==============================] - 0s 602us/step - loss: 8.4015e-04 - accuracy: 1.0000
Epoch 87/100
546/546 [==============================] - 0s 595us/step - loss: 0.0011 - accuracy: 0.9996
Epoch 88/100
546/546 [==============================] - 0

In [6]:
'''Predict with model'''

from keras.models import load_model
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))


def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [7]:
#Creating GUI with tkinter

import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "You: " + msg + '\n\n')
            ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
            res = chatbot_response(msg)
            ChatLog.insert(END, "Bot: " + res + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
base.mainloop()